# Samples and genome sequencing

Estimate phylogenetic tree with published short reads


In [ ]:
raxml-ng \
  --msa bernese_variable_positions.fasta \
  --all --model GTR+G --tree pars{10} --bs-trees 100 --threads 4

# Genome assembly and variant calling

The assembly pipeline is available as a Snakemake workflow on: http://git.scicore.unibas.ch/TBRU/PacbioSnake

Create a pangenome graph from the concatenated single contig assemblies

In [ ]:
pggb_latest.sif pggb \
  -i single_contig_assemblies.fasta.gz \
  -o ./pggb \
  -t 4 \
  -n 16 \
  -p 99 \
  -s 5k

Call variants from the pangenome graph, using N1426 as a positional reference

In [ ]:
pggb_latest.sif vg deconstruct \
  pggb/single_contig_assemblies.fasta.gz.*.smooth.final.gfa -d1 -e \
    -p N1426_1 \
    -t 4 \
    --all-snarls \
    > variants.vcf

# Assembly validation and curation

The alignment of long reads against the assembly built from them is part of the [assembly pipeline](https://git.scicore.unibas.ch/TBRU/PacbioSnake/-/blob/scicore/workflow/rules/mapreads.smk).

Call variants from the aligned long reads


In [ ]:
while read STRAIN; do

    ASSEMBLY=${STRAIN}.fasta
    READS=${STRAIN}/remapping/longreads.bam
    
    freebayes -p 1 -f ${ASSEMBLY} ${READS} | gzip > ${STRAIN}.var.vcf.gz

done < samples.txt

# Get  inconsistent sites (GT==1):
for VCF in *.vcf.gz; do 
    STRAIN=$(echo $VCF | cut -d'.' -f1)
    vcftools --gzvcf ${STRAIN}.var.vcf.gz --extract-FORMAT-info GT --stdout | awk '$3==1' >> inconsistent_sites.tsv
done

# Gene and repeat annotation

Gene annotation with bakta is part of the [assembly pipeline](https://git.scicore.unibas.ch/TBRU/PacbioSnake/-/blob/scicore/workflow/rules/annotate.smk).

Annotate insertion sequences with ISEScan


In [ ]:
isescan.py \
--seqfile ${ASSEMBLY} \
--output isescan \
--nthread 4

Annotate short sequence repeats (<= 9bp) with kmer-ssr

In [ ]:
kmer-ssr \
  -i ../$ASSEMBLY \
  -o $STRAIN.SSRs.tsv \
  -p 2-9 -r 3 -t 4

Annotate tandem repeats (>9bp) with SPADES

In [ ]:
SPADE.py -in $ASSEMBLY -n 4 -v Y

Annotate homopolymers

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
from Bio import SeqIO

try:
    fasta = sys.argv[1]
    min_len = int(sys.argv[2])
    
except IndexError:
    sys.exit("Usage: annotate_homopolymers.py <genome.fasta> <minimum tract length>. Writes to stdout.")

for seq_record in SeqIO.parse(fasta, "fasta"):
    
    contig = seq_record.id
    
    previous = ''
    counter = 0
    
    for i, base in enumerate(seq_record.seq):
        
        # initiate
        if base == previous and counter == 0:
            counter = 2
            
        # extent
        elif base == previous and counter > 0:
            counter += 1
        
        # terminate
        elif base != previous:
                if counter >= min_len:
                    outline = map(str, [contig, i-counter, i-1, counter, previous])
                    sys.stdout.write("\t".join(outline) +'\n')
                    #print(contig, i-counter, i-1, counter, previous)
                    counter = 0
                    
                else:
                    counter = 0
                    
        previous = base

Identify interspersed repeats

In [ ]:
nucmer --maxmatch --nosimplify $GENOME $GENOME
show-coords out.delta -Tcdl > delta.out.tsv

In [ ]:
# Filter delta out file

import seaborn
import pandas as pd

stats = {
    "overlap": 0,
    "identical" : 0,
    "keep" : 0
}

header = ['S1','E1','S2','E2','LEN1','LEN2', 'PID','LENR','LENQ','COVR','COVQ','RFRM', 'QFRM','TAG1', 'TAG2']

t = pd.read_csv("delta.out.tsv", skiprows=[0,1,2,3], sep='\t', names=header)

print(t.describe())
t.head()

# Remove self hits
start_diff = t['S1'] != t['S2']
end_diff = t['E1'] != t['E2']
t = t[start_diff | end_diff]

# Remove overlapping
overlap_filter = t['S2'] > t['E1']
t = t[overlap_filter]

# Remove dissimilar
id_filt = t['PID'] > 90
t = t[id_filt]

# Minimum length?
print(seaborn.displot(t['PID'], kind="kde"))
print(seaborn.displot(t['LEN1'], kind="kde"))
print(seaborn.displot(t, x="LEN1", y="PID", kind="kde", logx=True))

# Write filtered table
t.to_csv("delta.out.filtered.tsv", index=False)

Intersect annotations

In [ ]:
bedtools intersect -a bernese.variants.bed -b N1426.CDS.gff3 $INSERTIONSEQS $REPEATS $HPS $SSR $HOMSEGS -wao \
-names bakta insertion_sequences repeats homopolymers SSR homologysegments \
> bernese.variants.intersects.tsv

## Identification of gene conversion tracts

Extract conversion tracts

In [ ]:
OG=../B_call_variants/pggb/single_contig_assemblies.fasta.gz.d71a954.eb0f3d3.33064a1.smooth.final.og
GENOME=../../assembly/N1392/N1392.fasta

# Coordinates of suspected conversion tracts in the reference
FIRST_TRACT_POS=1640207
LAST_TRACT_POS=1640702

# Liftover coordinates to the genome in which the variants were called
pggb_latest.sif odgi position \
  -i $OG \
  -p N1426_1,$FIRST,+ \
  -r N1377_1 

pggb_latest.sif odgi position \
  -i $OG \
  -p N1426_1,$LAST,+ \
  -r N1377_1

# Get sequence of conversion tract
samtools faidx \
  $GENOME \
  N1392_1:$((1640207 - 5))-$((1640702 + 5)) > N1392.PE_PGRS28.conversion_tract.fasta


Blast conversion tracts

In [ ]:
blastn \
  -query N1392.PE_PGRS28.conversion_tract.fasta \
  -subject ../../assembly/N1392/N1392.fasta \
  -word_size 7 \
  -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore"

## Comparison with Illumina data

Get SNPs in the core genome from the pangenome graph

In [ ]:
variants = 'variants.vcf'  # variants called from the pangenome graph
reference = 'N1426_1'

keep = []

discarded = {
    "Not SNP" : 0,
    "Missing genotypes" : 0,
    "Invariant" : 0
}

with open(variants) as f:
    
    for line in f:
        
        if line.startswith("#CHROM"):
            
            fields = line.strip().split("\t")
            strains = fields[9:]

            # Dictionary to store SNPs for each strain
            strain_seqs = {strain : "" for strain in strains}
            strain_seqs[reference] = ""
            continue
        
        elif line.startswith("#"):
            continue

        fields = line.strip().split("\t")
        position = int(fields[1])
        
        ref = fields[3]
        alt = fields[4].split(",")
        
        gt = fields[9:]
        
        # Skip rows with missing genotypes
        if len(gt) != len(strains):
            print('Missing genotypes', fields)
            continue
        
        # Only SNPs
        if len(ref) > 1 or any(len(x) > 1 for x in alt):
            discarded["Not SNP"] += 1
            continue

        # Exclude missing genotypes
        if any(x in gt for x in ["", "."]):
            discarded["Missing genotypes"] += 1
            #continue

        # Write alleles to site
        alleles = [ref] + alt       
        site = ref
        
        for strain in strains:
            strain_i = strains.index(strain)
            strain_gt = gt[strain_i]
            if not strain_gt or strain_gt not in '012345':
                strain_allele = '-'
            else:
                strain_allele = alleles[int(strain_gt)]
            
            site += strain_allele
            
        # Re-check if there are remaining invariant sites
        bases_only = ''
        for allele in site:
            if allele in ['A','C','G', 'T']:
                bases_only += allele

        
        strains_with_ref = [reference] + strains
        
        if len(set(bases_only)) > 1:
                
            for i, allele in enumerate(site):
                strain = strains_with_ref[i]
                strain_seqs[strain] += allele
                            
            keep.append((position, alleles, gt))
            
        else:
            print('invariant:', fields)
            discarded["Invariant"] += 1

                
print("Filtered out:")
for k in discarded:
    print(k, str(discarded[k]))
    
print("Kept: ", str(len(keep)))

Write SNP alignment

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

records = []

for strain in strains_with_ref:
    sequence = Seq(strain_seqs[strain])
    record = SeqRecord(sequence, id=strain, name="", description="")
    records.append(record)

SeqIO.write(records, "SNP_alignment_from_graph.fasta", "fasta")

Create an alignment with all variants, artifically coding any variant as one of four bases. 

In [ ]:
import csv

vcf = 'variants.vcf'

# Ignore positions in grcC, which seems to be under convergent selection
ignore = [
    656926, 657255, 657265,657277, 657321, 657321, 657323  # grcC
    ]

# Load the VCF file
with open(vcf, 'r') as vcf_file:
    vcf_reader = csv.reader(vcf_file, delimiter='\t')
    # Skip the header lines
    for _ in range(7):
        next(vcf_reader)

    # Extract the sample IDs, add reference
    sample_ids = next(vcf_reader)[9:]
    sample_ids.insert(0,'N1426_1')

    # Initialize a dictionary to store the genotypes
    genotypes = {sample_id: [] for sample_id in sample_ids}
    genotypes_filtered = {sample_id: [] for sample_id in sample_ids}
    
    # Create artificial alignment for distance tree estimation
    aln_artif = {sample_id: [] for sample_id in sample_ids} 
    aln_artif_filtered = {sample_id: [] for sample_id in sample_ids} 
    
    bases = {
        '0':'A',
        '1':'C',
        '2':'G',
        '3':'T'
    }
    
    # Iterate over the variants
    for row in vcf_reader:
        
        pos = int(row[1])
        
        # Extract the genotypes
        genotypes_row = ['0'] + row[9:]
        for sample_id, genotype in zip(sample_ids, genotypes_row):
            genotypes[sample_id].append(genotype)
            aln_artif[sample_id].append(bases[genotype])
            
            if pos not in ignore:
                genotypes_filtered[sample_id].append(genotype)
                aln_artif_filtered[sample_id].append(bases[genotype])

Write sequences

In [ ]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

outhandle = open('trees/pacbio/artificial_alignment.all_variants.fasta', 'w')
outhandle_filtered = open('trees/pacbio/artificial_alignment.all_variants.filtered.fasta', 'w')

# Write artifical alignment
for k in aln_artif:
    
    SeqIO.write(
        SeqRecord(Seq(''.join(aln_artif[k])), id=k, name="", description=""), 
        outhandle, "fasta")
    
    SeqIO.write(
        SeqRecord(Seq(''.join(aln_artif_filtered[k])), id=k, name="", description=""), 
        outhandle_filtered, "fasta")
    
outhandle.close()
outhandle_filtered.close()